In [6]:
from pathlib import Path
import sys

PROJECT_ROOT = Path().resolve().parents[1]
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import numpy as np
import pandas as pd
import librosa
from scipy.io import wavfile

from Code.audio import AudioPreproc, AudioPreprocConfig, AudioFeat, AudioFeatConfig
from Code.audio.Standardizer import Standardizer

run = 3

input_dir_base = PROJECT_ROOT / "Database" / "data" / "audio"
output_dir_base = PROJECT_ROOT / "Database" / "tmp" / f"audioSTDTry{run:02d}"
output_dir_base.mkdir(parents=True, exist_ok=True)

labels = ["contar", "proporcion", "salir"]

pre_config = AudioPreprocConfig(
    target_sr=16000, t_sec=1.2,
    frame_ms=25.0, hop_ms=10.0,
    highpass_hz=40.0, hp_order=2,
    preemph_a=0.97,
    vad_thresh_db=-35.0, vad_win_ms=20.0,
    vad_min_ms=120.0, vad_expand_ms=60.0,
    norm_mode="rms", rms_target_dbfs=-20.0,
    peak_ref=0.98, max_gain_db=18.0,
    gate_dbfs=-60.0, pad_mode="edge",
)
pre = AudioPreproc(pre_config)

winn, hopp = pre.cfg.frame_ms*1e-3, pre.cfg.hop_ms*1e-3

feat_config = AudioFeatConfig(
    win=winn,
    hop=hopp,
    n_mfcc_no_c0=13,
    delta_order=1,
    add_rms=True,
    add_zcr=True,
    stats=("mean", "std", "p10", "p90"),
)
feat = AudioFeat(cfg=feat_config)
feature_names = feat.nombres_features()

y_proc, sr_proc, file_names, class_names = [], [], [], []
y_features = []

for cls in labels:
    input_dir = input_dir_base / cls
    output_dir = output_dir_base / cls
    output_dir.mkdir(parents=True, exist_ok=True)

    for audio_path in sorted(input_dir.glob("*.wav")):
        y, sr = pre.preprocesar_desde_path(audio_path)
        y_feat = feat.extraer_caracteristicas(y, pre_config.target_sr)

        y_proc.append(y)
        sr_proc.append(sr)
        y_features.append(y_feat)
        
        file_names.append(audio_path.name)
        class_names.append(cls)


# Tabla de estadísticos básicos por archivo (opcional)
stats_rows = []
for cls, fname, y, sr in zip(class_names, file_names, y_proc, sr_proc):
    stats_rows.append({
        "Clase": cls,
        "Archivo": fname,
        "Duración (s)": len(y) / sr,
        "Mín": float(np.min(y)),
        "Máx": float(np.max(y)),
        "Energía RMS": float(np.sqrt(np.mean(y**2))),
    })
df_stats = pd.DataFrame(stats_rows)
df_stats.to_csv(output_dir_base / "estadisticos.csv", index=False)

# Tabla de features completas por archivo
feat_rows = []
for cls, fname, vec in zip(class_names, file_names, y_features):
    row = {"Clase": cls, "Archivo": fname}
    row.update({name: float(val) for name, val in zip(feature_names, vec)})
    feat_rows.append(row)

df_feat = pd.DataFrame(feat_rows)
df_feat.to_csv(output_dir_base / "features.csv", index=False)

df_feat  # muestra la tabla de features en el notebook

# Covarianza y reducción de dimensión sobre las features

X = np.stack(y_features).astype(np.float32)  # (N, D)
print(f"Dimensión de matriz de features X: {X.shape}")

stats = Standardizer().calculate_statistics(X)
X_std = stats.transform(X)

print(f"Dimensión de matriz X_std: {X_std.shape}")

cov = np.cov(X_std, rowvar=False)
eigvals, eigvecs = np.linalg.eigh(cov)
idx = np.argsort(eigvals)[::-1]
eigvals = eigvals[idx]
eigvecs = eigvecs[:, idx]

explained = eigvals / eigvals.sum()
cum_explained = np.cumsum(explained)
k_95 = int(np.searchsorted(cum_explained, 0.95) + 1)

explained *= 100.0
cum_explained *= 100.0

# Proyección en las k componentes que cubren ~95% de la varianza
X_proj = X_std @ eigvecs[:, :k_95]

cov_df = pd.DataFrame(cov, index=feature_names, columns=feature_names)
var_df = pd.DataFrame({
    'eigenvalue': eigvals,
    'Indice': idx,
    'Porcentaje': explained,
    'Porcentaje Acumulado': cum_explained,
})
loadings_df = pd.DataFrame(
    eigvecs[:, :k_95],
    index=feature_names,
    columns=[f'PC{i+1}' for i in range(k_95)],
)

display(var_df.head(20))
display(loadings_df.head(20))

# Guardar resultados en CSV en la misma carpeta de salida
cov_df.to_csv(output_dir_base / 'covariance.csv')
var_df.assign(pc=np.arange(1, len(eigvals)+1)).to_csv(output_dir_base / 'variance_explained.csv', index=False)
loadings_df.to_csv(output_dir_base / 'loadings.csv')
proj_rows = []

for fname, cls, vec in zip(file_names, class_names, X_proj):
    row = {'Archivo': fname, 'Clase': cls}
    row.update({f'PC{i+1}': float(val) for i, val in enumerate(vec)})
    proj_rows.append(row)

pd.DataFrame(proj_rows).to_csv(output_dir_base / 'projections.csv', index=False)


Dimensión de matriz de features X: (33, 112)
Dimensión de matriz X_std: (33, 112)


,eigenvalue,Indice,Porcentaje,Porcentaje Acumulado
0,28.540172,111,24.932719,24.932719
1,15.306608,110,13.371866,38.304585
2,10.710541,109,9.356737,47.661322
3,8.879256,108,7.756926,55.418248
4,7.007056,107,6.121370,61.539618
5,5.714847,106,4.992495,66.532114
6,5.536641,105,4.836814,71.368928
7,4.410750,104,3.853235,75.222163
8,3.388552,103,2.960242,78.182405
9,3.079289,102,2.690070,80.872474


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20
mfcc_1_mean,0.085499,-0.098172,0.030644,-0.244752,0.065144,0.023920,-0.023076,0.007707,-0.011764,0.104906,-0.078627,0.077178,-0.029683,0.052853,-0.013283,0.065089,-0.034608,0.054396,-0.009136,0.036908
mfcc_2_mean,0.149777,-0.141044,0.023209,0.035335,0.032042,-0.050834,-0.002068,-0.032915,0.043033,-0.031261,-0.004677,-0.034692,-0.066758,0.013216,-0.027976,0.049226,0.012277,0.025716,0.028310,-0.034240
mfcc_3_mean,0.138133,-0.145441,0.023501,0.096708,0.026888,0.027697,-0.017510,-0.009760,-0.020217,-0.063818,0.006986,-0.002905,-0.006133,-0.012297,0.001086,-0.091833,-0.055557,0.041223,-0.106337,-0.009565
mfcc_4_mean,0.160388,-0.060441,-0.007997,0.069018,-0.094025,-0.035765,-0.042493,0.095834,-0.009707,0.038888,-0.076484,0.006003,-0.059404,0.026779,0.003763,-0.018429,0.122432,0.053371,-0.108550,-0.000523
mfcc_5_mean,0.150786,-0.002977,-0.136395,-0.060104,-0.058637,0.014155,-0.109845,0.009065,-0.003150,-0.060659,0.035949,0.008892,0.018183,-0.035439,-0.012092,-0.021720,-0.007650,0.070111,-0.048797,-0.017961
mfcc_6_mean,0.176681,-0.020430,0.034050,-0.047069,0.017063,0.005383,0.059791,0.056891,0.058807,-0.070392,-0.021798,-0.040841,0.028083,0.066846,-0.046861,-0.029699,-0.051767,-0.017804,-0.031155,-0.034743
mfcc_7_mean,0.144538,-0.046276,0.098476,0.021156,-0.005909,-0.034449,-0.025227,-0.144312,0.157820,-0.009467,0.048079,0.068890,-0.098717,0.027478,0.002024,-0.046120,0.020166,-0.110392,-0.130919,-0.025055
mfcc_8_mean,0.164775,0.064285,-0.078122,0.035466,-0.013034,-0.033516,0.046420,-0.004234,0.015956,0.061878,-0.012890,0.043040,-0.030378,0.054602,-0.081872,-0.129673,0.019446,-0.037293,-0.014314,-0.042593
mfcc_9_mean,0.040729,0.021136,0.254165,-0.030772,-0.079517,0.073833,0.049806,0.082295,0.006666,-0.011906,-0.146236,0.023405,-0.104336,-0.033689,-0.016339,-0.022210,0.027004,0.124871,-0.122897,-0.087407
mfcc_10_mean,0.121296,0.047561,-0.048196,0.026763,0.123186,-0.174717,0.087828,-0.030951,-0.016205,0.133927,-0.063454,-0.065269,0.094570,0.029756,-0.007802,-0.198537,-0.113636,-0.043644,-0.061841,-0.067255
